In [1]:
import yfinance as yf
from matplotlib import pyplot as plt

Get SP500 history using the index and Ticker Object

In [2]:
sp500 = yf.Ticker("^GSPC")

In [3]:
sp500 = sp500.history(period="max")

In [5]:
sp500

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-22 00:00:00-04:00,5832.700195,5863.040039,5821.169922,5851.200195,3342080000,0.0,0.0
2024-10-23 00:00:00-04:00,5834.500000,5834.850098,5762.410156,5797.419922,3532650000,0.0,0.0
2024-10-24 00:00:00-04:00,5817.799805,5817.799805,5784.919922,5809.859863,3543030000,0.0,0.0


Remove last two columns because they do not apply to an Index Fund

In [8]:
del sp500["Dividends"]
del sp500["Stock Splits"]

Add and fill new column, Tomorrow

In [9]:
sp500["Tomorrow"] = sp500["Close"].shift(-1)

Add and fill new column, Target (will tomorrow's price increase)

In [10]:
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)

Avoid historically irrelevant data

In [11]:
sp500 = sp500.loc["1990-01-01":].copy()

Random Forest b/c they can represent non-linear tendencies and are not prone to over-fitting (b/c averaging)

In [14]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
        n_estimators=100, # number of trees
        min_samples_split=100,  #  how deeply each tree is allowed to grow (higher = less accurate, less overfit)
        random_state=1  # random seed for repeadability w/ testing
        )

# train/test sets, training excluding the last 100 days to prevent leakage
train = sp500.iloc[:-100].copy()
test = sp500.iloc[-100:].copy()

predictors = ["Close", "Volume", "Open", "High", "Low"]
model.fit(train[predictors], train["Target"])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [ ]:
from sklearn.metrics import precision_score
import pandas as pd

predictions = model.predict(test[predictors])

predictions = pd.Serires(predictions, index=test.index)